# Traffic Easing Dataset Analysis
- speed bumps
- speed limit

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely import wkt
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

## Speed Bumps

In [75]:
speed_bump = pd.read_csv('./crash_matched_data/speed_bumps_matched.csv')

In [76]:
speed_bump.head()

,date_date_,time_date_,from_stree,new_humps,on_street,shape_leng,to_street,geometry,index_right,Unnamed: 0,Roadway.Name,From,To,int1,int2,zip1,zip2,ll1,ll2,lat1,long1,lat2,long2,top_lat,right_long,bot_lat,left_long,botright_lat,botright_long,botleft_lat,botleft_long,topright_lat,topright_long,topleft_lat,topleft_long,streetID
0,2014-05-22,00:00:00.000,6 AVENUE,2.0,1 STREET,775.659477,7 AVENUE,MULTIPOINT (-73.9799168591299 40.6728580233743...,126.0,148.0,3 AVENUE,29 STREET,30 STREET,3+AVENUE+%26+29+STREET,3+AVENUE+%26+30+STREET,11232,10016.0,"40.659353,-74.002185","40.743208,-73.979949",40.659353,-74.002185,40.743208,-73.979949,40.743208,-73.979949,40.659353,-74.002185,40.659223,-73.979819,40.659223,-74.002315,40.743338,-73.979819,40.743338,-74.002315,148.0
1,2014-05-22,00:00:00.000,6 AVENUE,2.0,1 STREET,775.659477,7 AVENUE,MULTIPOINT (-73.9799168591299 40.6728580233743...,129.0,151.0,3 AVENUE,EAST 60 STREET,61 STREET,3+AVENUE+%26+EAST+60+STREET,3+AVENUE+%26+61+STREET,10022,11220.0,"40.762168,-73.966129","40.641597,-74.021248",40.762168,-73.966129,40.641597,-74.021248,40.762168,-73.966129,40.641597,-74.021248,40.641467,-73.965999,40.641467,-74.021378,40.762298,-73.965999,40.762298,-74.021378,151.0
2,2017-09-11,00:00:00.000,HOYT STREET,2.0,1 STREET,652.672074,BOND STREET,MULTIPOINT (-73.99294800158468 40.678995874101...,126.0,148.0,3 AVENUE,29 STREET,30 STREET,3+AVENUE+%26+29+STREET,3+AVENUE+%26+30+STREET,11232,10016.0,"40.659353,-74.002185","40.743208,-73.979949",40.659353,-74.002185,40.743208,-73.979949,40.743208,-73.979949,40.659353,-74.002185,40.659223,-73.979819,40.659223,-74.002315,40.743338,-73.979819,40.743338,-74.002315,148.0
3,2017-09-11,00:00:00.000,HOYT STREET,2.0,1 STREET,652.672074,BOND STREET,MULTIPOINT (-73.99294800158468 40.678995874101...,188.0,225.0,5 AVENUE,30 STREET,31 STREET,5+AVENUE+%26+30+STREET,5+AVENUE+%26+31+STREET,11232,10001.0,"40.656320,-73.998669","40.746539,-73.985973",40.656320,-73.998669,40.746539,-73.985973,40.746539,-73.985973,40.656320,-73.998669,40.656190,-73.985843,40.656190,-73.998799,40.746669,-73.985843,40.746669,-73.998799,225.0
4,2017-09-11,00:00:00.000,HOYT STREET,2.0,1 STREET,652.672074,BOND STREET,MULTIPOINT (-73.99294800158468 40.678995874101...,129.0,151.0,3 AVENUE,EAST 60 STREET,61 STREET,3+AVENUE+%26+EAST+60+STREET,3+AVENUE+%26+61+STREET,10022,11220.0,"40.762168,-73.966129","40.641597,-74.021248",40.762168,-73.966129,40.641597,-74.021248,40.762168,-73.966129,40.641597,-74.021248,40.641467,-73.965999,40.641467,-74.021378,40.762298,-73.965999,40.762298,-74.021378,151.0


In [80]:
speed_bump.time_date_.value_counts()

00:00:00.000    4384
Name: time_date_, dtype: int64

In [83]:
bump_loc = speed_bump[["date_date_", "from_stree", "new_humps", "geometry", "lat1", "long1", "Roadway.Name"]].dropna()

In [84]:
#map coordinates 
map = folium.Map(location=[bump_loc.lat1.mean(), 
                           bump_loc.long1.mean()], zoom_start=14, control_scale=True)

for index, location_info in bump_loc.iterrows(): 
    folium.Marker([location_info["lat1"], location_info["long1"]], popup=location_info["Roadway.Name"]).add_to(map)

In [85]:
map

# Speed Limits

In [87]:
speed_limits = pd.read_csv('./crash_matched_data/speed_limits_matched.csv')

/var/folders/ks/gjt8vnfs62sc_5gmd7sdvxph0000gn/T/ipykernel_57138/1106378054.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  speed_limits = pd.read_csv('./crash_matched_data/speed_limits_matched.csv')


In [89]:
#create new df for mapping with required cols
speed_limits = speed_limits[['postvz_sg', 'postvz_sl', 'street', 'shape_leng', 'geometry', 'streetID']].dropna()
speed_limits['geometry'] = speed_limits['geometry'].astype(str).apply(wkt.loads)
speed_limits = gpd.GeoDataFrame(speed_limits, geometry = 'geometry')                    

In [90]:
speed_limits.head()

,postvz_sg,postvz_sl,street,shape_leng,geometry,streetID
12,NO,25.0,TRAVIS AVENUE,637.659701,"MULTIPOINT (-74.16562 40.59362, -74.16778 40.5...",2189.0
13,NO,25.0,TRAVIS AVENUE,637.659701,"MULTIPOINT (-74.16562 40.59362, -74.16778 40.5...",2188.0
16,NO,25.0,EAST MOSHOLU PARKWAY SOUTH,350.205319,"MULTIPOINT (-73.88340 40.87395, -73.88296 40.8...",971.0
17,NO,25.0,EAST MOSHOLU PARKWAY SOUTH,350.205319,"MULTIPOINT (-73.88340 40.87395, -73.88296 40.8...",1721.0
24,NO,25.0,EVERGREEN AVENUE,242.154925,"MULTIPOINT (-73.93423 40.70330, -73.93395 40.7...",1181.0
